In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import src.config as config 

In [4]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)

2024-11-29 19:22:38,152 INFO: Initializing external client
2024-11-29 19:22:38,153 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-11-29 19:22:41,511 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1018005


In [5]:
# create feature view (if it doesn't exist yet)
# This feature view only uses on feature group, so the query is trivial
try:
    # create feature view if it doesn't exist yet
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print('Feature view already existed. Skip creation.')


# get feature view
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)

Feature view already existed. Skip creation.


In [6]:
ts_data, _ = feature_view.training_data(
    description='Time-series hourly taxi rides',
)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (668.74s) 


In [ ]:
# drop `pickup_ts` column
ts_data.drop('pickup_ts', axis=1, inplace=True)

# sort by `pickup_location_id` and `pickup_hour`
ts_data.sort_values(by=['pickup_location_id', 'pickup_hour'], inplace=True)
ts_data


In [ ]:
# from src.plot import plot_ts
from typing import Optional, List
import pandas as pd
import plotly.express as px 

def plot_ts(
    ts_data: pd.DataFrame,
    locations: Optional[List[int]] = None
    ):
    """
    Plot time-series data
    """
    ts_data_to_plot = ts_data[ts_data.pickup_location_id.isin(locations)] if locations else ts_data

    fig = px.line(
        ts_data_to_plot,
        x="pickup_hour",
        y="rides",
        color='pickup_location_id',
        template='none',
    )

    fig.show()

plot_ts(ts_data, locations=[43])

In [ ]:
from src.data import transform_ts_data_into_features_and_target

features, targets = transform_ts_data_into_features_and_target(
    ts_data,
    input_seq_len=24*28, # one month
    step_size=23,
)

features_and_target = features.copy()
features_and_target['target_rides_next_hour'] = targets

print(f'{features_and_target.shape=}')

100%|██████████| 259/259 [00:03<00:00, 75.53it/s]

features_and_target.shape=(1286, 675)


In [ ]:
features_and_target

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,1.0,0.0,0.0,0.0,1.0,3.0,1.0,2.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,2024-11-25 10:00:00+00:00,1,2.0
1,0.0,0.0,0.0,2.0,0.0,6.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2024-11-26 09:00:00+00:00,1,0.0
2,1.0,1.0,0.0,1.0,0.0,1.0,1.0,2.0,2.0,3.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,2024-11-27 08:00:00+00:00,1,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,1.0,4.0,4.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2024-11-28 07:00:00+00:00,1,0.0
4,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2024-11-29 06:00:00+00:00,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1281,3.0,7.0,4.0,5.0,3.0,1.0,3.0,0.0,2.0,1.0,...,1.0,1.0,4.0,4.0,2.0,2.0,1.0,2024-11-25 10:00:00+00:00,265,2.0
1282,1.0,3.0,6.0,1.0,1.0,3.0,0.0,2.0,4.0,3.0,...,0.0,0.0,0.0,0.0,7.0,2.0,0.0,2024-11-26 09:00:00+00:00,265,1.0
1283,1.0,2.0,4.0,1.0,1.0,0.0,4.0,2.0,2.0,0.0,...,0.0,2.0,0.0,0.0,3.0,3.0,0.0,2024-11-27 08:00:00+00:00,265,3.0
1284,0.0,2.0,0.0,3.0,2.0,2.0,2.0,3.0,1.0,2.0,...,5.0,7.0,2.0,1.0,0.0,1.0,2.0,2024-11-28 07:00:00+00:00,265,2.0


: 

In [ ]:
from datetime import date, timedelta
from pytz import timezone
import pandas as pd
from src.data_split import train_test_split

# training data -> from January 2022 up until 2 months ago
# test data -> last 2 months
cutoff_date = pd.to_datetime(date.today() - timedelta(days=28*1), utc=True)

print(f'{cutoff_date=}')

features_and_target["pickup_hour"] = pd.to_datetime(features_and_target["pickup_hour"])

X_train, y_train, X_test, y_test = train_test_split(
    features_and_target,
    cutoff_date,
    target_column_name='target_rides_next_hour'   
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

cutoff_date=Timestamp('2024-11-01 00:00:00+0000', tz='UTC')


NameError: name 'features_and_target' is not defined

In [ ]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from src.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
       
    tss = KFold(n_splits=3)
    scores = []
        
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
         
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)

[I 2024-11-29 19:12:33,427] A new study created in memory with name: no-name-e12df55f-8297-4c32-b220-7ec4926b6679
[W 2024-11-29 19:12:33,434] Trial 0 failed with parameters: {'num_leaves': 64, 'feature_fraction': 0.38536333111967114, 'bagging_fraction': 0.8836952017858684, 'min_child_samples': 42} because of the following error: ValueError('Cannot have number of splits n_splits=3 greater than the number of samples: n_samples=0.').
Traceback (most recent call last):
  File "d:\Real_world_Ml_Project\taxi_demand_predictor\.venv\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\HP\AppData\Local\Temp\ipykernel_22188\436714379.py", line 27, in objective
    for train_index, val_index in tss.split(X_train):
  File "d:\Real_world_Ml_Project\taxi_demand_predictor\.venv\lib\site-packages\sklearn\model_selection\_split.py", line 370, in split
    raise ValueError(
ValueError: Cannot have number of splits n_splits=3 greater tha

ValueError: Cannot have number of splits n_splits=3 greater than the number of samples: n_samples=0.

: 

In [ ]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 150, 'feature_fraction': 0.4576721724346207, 'bagging_fraction': 0.990878647061735, 'min_child_samples': 45}


In [ ]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.4576721724346207, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4576721724346207
[LightGBM] [Warning] bagging_fraction is set=0.990878647061735, subsample=1.0 will be ignored. Current value: bagging_fraction=0.990878647061735
[LightGBM] [Warning] feature_fraction is set=0.4576721724346207, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4576721724346207
[LightGBM] [Warning] bagging_fraction is set=0.990878647061735, subsample=1.0 will be ignored. Current value: bagging_fraction=0.990878647061735
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.121266 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 171876
[LightGBM] [Info] Number of data points in the train set: 211288, number of used features: 676
[LightGBM] [Info] Start training from score 1

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x0000024A25013820>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.990878647061735,
                               feature_fraction=0.4576721724346207,
                               min_child_samples=45, num_leaves=150))])

In [ ]:
predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

[LightGBM] [Warning] feature_fraction is set=0.4576721724346207, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4576721724346207
[LightGBM] [Warning] bagging_fraction is set=0.990878647061735, subsample=1.0 will be ignored. Current value: bagging_fraction=0.990878647061735
test_mae=4.7215


In [ ]:
import joblib
from src.paths import MODELS_DIR

joblib.dump(pipeline, MODELS_DIR / 'model.pkl')

['D:\\Real_world_Ml_Project\\taxi_demand_predictor\\models\\model.pkl']

In [ ]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

In [ ]:
model_registry = project.get_model_registry()


model = model_registry.sklearn.create_model(
    name="taxi_demand_predictor_next_hour",
    metrics={"test_mae": test_mae},
    description="LightGBM regressor with a bit of hyper-parameter tuning",
    input_example=X_train.sample(),
    model_schema=model_schema
)

model.save(str(MODELS_DIR / 'model.pkl'))

Connected. Call `.close()` to terminate connection gracefully.


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/1362610 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/3404 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/60849 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1018005/models/taxi_demand_predictor_next_hour/2


Model(name: 'taxi_demand_predictor_next_hour', version: 2)